'''


    @Author: Prathamesh Deshpande
    @Date: 21/11/2024
    @Last Modified by: Prathamesh Deshpande
    @Last Modified time: 21/11/2024
    @Title: python Code To summarize the mail and translate into Marathi using prompt engineering


'''

Importing Libraries

In [1]:
import os
import google.generativeai as genai
import csv

Config Gemini AI

In [2]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

generation_config = {
  "temperature": 0.9,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

Reading Mail information from Sample_Mails.txt File

In [3]:
def read_emails(file_path, delimiter="---END OF EMAIL---"):
    """
    Description: 
        This function reads the email from the text file

    Parameters:
        file_path: Path of txt file
        delimiter: delimiter

    Return:
        list: list of emails
    """
    
    with open(file_path, 'r') as file:
        emails = file.read().split(delimiter)
        
    return [email.strip() for email in emails if email.strip()]  # Return non-empty emails

In [4]:
def extract_email_info(email):
    """
    Description: 
        This Function extracts sender, receiver, and body from email

    Parameters:
        email
        
    Return:
        sender, receiver, and body (str)
    """
    lines = email.split("\n")
    sender = receiver = body = subject = ""
    
    for line in lines:
        if line.startswith("From:"):
            sender = line.split(":")[1].strip()

        elif line.startswith("To:"):
            receiver = line.split(":")[1].strip()

        elif line.startswith("Subject:"):
            subject = line.split(":")[1].strip()

        elif not line.startswith("Subject:") and not line.startswith("From:") and not line.startswith("To:"):
            body += line.strip() + " "
    
    return sender, receiver,subject, body.strip()

Summarizing Task

In [5]:
def summarize_email(body, chat_session):
    """
    Description: 
        This Function summarizes email body using the Gemini model

    Parameters:
        body: body of email
        chat_session: gemini ai chat session

    Return:
        response(str): summarized response
    """
    
    response = chat_session.send_message(f"Summarize the following email: {body}")
    return response.text

Translation Task

In [6]:
def translate_email(text, chat_session):
    """
    Description: 
        This Function translates text using the Gemini model

    Parameters:
        body: body of email
        chat_session: gemini ai chat session

    Return:
        response(str): summarized response
    """
    
    response = chat_session.send_message(f"Translate the following text to Marathi: {text}")
    return response.text

Save output to csv file

In [7]:
def save_to_csv(data, csv_file):
    """
    Description: 
        This function save the response in csv file
    Parameters:
        data: data to be saved in csv
        csv_file: csv file name/path
    """
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Sender", "Receiver","Subject","Summary (EN)", "Summary (Marathi)"])
        writer.writerows(data)

In [8]:
def process_emails(file_path, csv_file, chat_session, delimiter="---END OF EMAIL---"):
    """
    Description: 
        This function processes the email 

    Parameters:
        file_path: file of email texts
        csv_file: name/path of csv file
        chat_session: gemini ai chat session
        
    Return:
        response(str): summarized response
    """
    emails = read_emails(file_path, delimiter)
    data = []

    for email in emails:
        sender, receiver, subject, body = extract_email_info(email)
        summary_en = summarize_email(body, chat_session)
        summary_marathi = translate_email(summary_en, chat_session)
        data.append([sender, receiver,subject ,summary_en, summary_marathi])
    
    save_to_csv(data, csv_file)

In [9]:
def main():
    chat_session = model.start_chat(
        history=[]
    )
    process_emails('Sample_Emails.txt', 'Processed_emails.csv', chat_session)

In [10]:
if __name__ == '__main__':
    main()